# Connect to DB, Fetch data

In [35]:
import pandas as pd
# specify connection to database
import psycopg2
connection = psycopg2.connect(
    host="nursedash-prod.cuzi2kducsnv.us-east-1.rds.amazonaws.com",
    database="nursedash",
    user="external_analyst",
    password="uDps8APganhSLc3K2xe7NtMPq")

### <font color = green> all time to chicago time, No withdrawn info

In [36]:
df = pd.read_sql_query("""

SELECT  sa.id, sa.user_id, sa.shift_id, f.id AS facility_id, sa."withdrawnInfo" -> 'initiator' as withdrawnInfo_value,
sa."status", sa."prevStatus", sa."distance", s."facility_id", "s"."description" AS "shift_description",
"s"."assigned_nurse_id", s."net_pay", "s"."unit" AS "s_unit",s."type",
"s"."qualifications" AS "s_qualifications", "s"."breakTime" AS "s_breakTime", sa."withdrawnInfo",
"f"."name" AS "f_name","f"."short_name" AS "f_short_name", f."segmentName", f."areaName",
timezone('America/Chicago', s."createdAt") as s_create,
timezone('America/Chicago', sa."createdAt") as sa_create,
timezone('America/Chicago', u."approvedAt") as u_approve,
timezone('America/Chicago', u."createdAt") as u_create,
timezone('America/Chicago', sa."statusUpdatedAt") as sa_statusUpdate,
timezone('America/Chicago', timezone('UTC', s.start_time)) AS "Start_Time" 
FROM shifts s
INNER JOIN shift_applications sa ON s.id = sa.shift_id
INNER JOIN facilities f ON s.facility_id = f.id
INNER JOIN users u ON sa.user_id = u.id

""", con = connection)

In [37]:
df.columns

Index(['id', 'user_id', 'shift_id', 'facility_id', 'withdrawninfo_value',
       'status', 'prevStatus', 'distance', 'facility_id', 'shift_description',
       'assigned_nurse_id', 'net_pay', 's_unit', 'type', 's_qualifications',
       's_breakTime', 'withdrawnInfo', 'f_name', 'f_short_name', 'segmentName',
       'areaName', 's_create', 'sa_create', 'u_approve', 'u_create',
       'sa_statusupdate', 'Start_Time'],
      dtype='object')

### 2 new columns to determine target

In [38]:
# CW_Time2Start_Time
df['CW_Time2Start_Time'] = df.apply(lambda row: (row['Start_Time'] - row['sa_statusupdate']).total_seconds()/3600 if str(row['Start_Time'])[:2] != '00' else 9999, 
                                         axis = 1)

# Target
def CW_by_nurse(row):
    if row['status']=='withdrawn' and row['prevStatus'] == 'confirmed':
        if row['withdrawninfo_value'] == 'nurse':
            if row['CW_Time2Start_Time'] <= 24:
                return "UCW"
            else:
                return "CW"
        else:
            return 0
    else:
        return 0
    
df['target'] = df.apply (lambda row: CW_by_nurse(row), axis=1)

In [39]:
test = df[df['target']=="UCW"]
df[df['id']==172013]

,id,user_id,shift_id,facility_id,withdrawninfo_value,status,prevStatus,distance,facility_id,shift_description,...,segmentName,areaName,s_create,sa_create,u_approve,u_create,sa_statusupdate,Start_Time,CW_Time2Start_Time,target
172142,172013,11143,126274,495,nurse,withdrawn,confirmed,11085.0,495,"COVID-19 Unit. If selected, you will be workin...",...,Senior Living,DFW,2021-03-26 11:26:49.997509,2021-03-26 11:27:08.068697,2020-08-10 21:32:09.478,2020-08-10 19:38:17.666270,2021-04-05 09:29:02.034870,2021-04-27 18:00:00,536.516101,CW


# combine the prediction file with real data

In [41]:
# read the prediction file
prediction = pd.read_csv('pred_2021-4-4.csv')
validation = prediction.merge(df, on = 'id', how = 'left')

In [42]:
prediction

,id,Start_Time,prob
0,172321,2021-04-04 05:00:00,0.271126
1,170592,2021-04-04 05:00:00,0.415821
2,173405,2021-04-04 06:00:00,0.607138
3,172482,2021-04-04 06:00:00,0.468472
4,174072,2021-04-04 06:00:00,0.681695
...,...,...,...
801,170607,2021-05-01 05:00:00,0.156116
802,173937,2021-05-02 05:00:00,0.250273
803,170735,2021-05-02 06:00:00,0.393660
804,172969,2021-05-02 17:00:00,0.110990


In [43]:
from datetime import date

today = date.today()

# convert to datetime for conditonal selection
validation['Start_Time_x'] = pd.to_datetime(validation['Start_Time_x'])

# record as realdata
validation['Study Set'] = validation['Start_Time_x'].apply(lambda x: "Y" if x <= today else "N")

In [44]:
validation = validation.rename(columns={"Start_Time_x": "Start_Time"})
validation = validation[['id','prob','target','Study Set','Start_Time','status','type','prevStatus', 'withdrawnInfo','areaName','segmentName']]

In [45]:
validation = validation.set_index("id")
validation.columns

Index(['prob', 'target', 'Study Set', 'Start_Time', 'status', 'type',
       'prevStatus', 'withdrawnInfo', 'areaName', 'segmentName'],
      dtype='object')

In [46]:
# read excel and save to a particular worksheet & it will help update the pivot table
import pandas

from openpyxl import load_workbook

book = load_workbook('analysis-for-predictatiion_2021-4-4.xlsx')

writer = pandas.ExcelWriter('analysis-for-predictatiion_2021-4-4.xlsx', engine='openpyxl') 

writer.book = book

writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

validation.to_excel(writer, sheet_name='validation data')

writer.save()